# Medical Chatbot Experiment - Simple Colab Version

This is a simplified version that fixes the import errors.

**Setup:**
1. Runtime → Change runtime type → GPU
2. Run cells in order

## 1. Clone Repo & Fix Python Path

In [ ]:
# Clone repository (update URL with your repo)
!git clone https://github.com/yourusername/health-experiment.git
%cd health-experiment

# CRITICAL: Fix Python path so imports work
import sys
from pathlib import Path

project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("✓ Setup complete")
print(f"  Directory: {project_root}")
print(f"  src exists: {(project_root / 'src').exists()}")

## 2. Install Dependencies

In [ ]:
# Install PyTorch and LLM libraries
!pip install -q torch torchvision torchaudio
!pip install -q transformers>=4.35.0 accelerate>=0.24.0
!pip install -q bitsandbytes>=0.41.0 sentencepiece>=0.1.99
!pip install -q pyyaml numpy pandas python-dateutil tqdm

print("✓ Dependencies installed")

## 3. Check GPU

In [ ]:
import torch

print("=" * 60)
print("GPU CHECK")
print("=" * 60)
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n✓ GPU ready for LLMs!")
else:
    print("\n⚠️  No GPU detected!")
    print("Go to: Runtime → Change runtime type → GPU")

## 4. Test Import (Should Work Now!)

In [ ]:
from src.core.simulation import MedicalChatbotSimulation

print("✓ Import successful!")
print("  All src modules are accessible")

## 5. Initialize Simulation with LLMs

In [ ]:
print("Initializing simulation...")
print("(First time: downloads models, takes 5-10 minutes)\n")

sim = MedicalChatbotSimulation(
    config_path="config/experiment_config.yaml",
    random_seed=42
)

print("\n✓ Simulation initialized!")
print(f"  Patient LLM: {sim.patient_model_name or 'rule-based'}")
print(f"  Doctor LLM: {sim.doctor_model_name or 'rule-based'}")

## 6. Quick Test (1 day)

In [ ]:
print("Running quick test: 1 day, without history\n")

results = sim.run_experiment(
    patient_id="patient_001",
    num_days=1,
    condition="without_history",
    random_seed=42
)

print(f"\n✓ Test complete!")
print(f"  Conversations: {results['total_conversations']}")
print(f"  Total turns: {results['total_turns']}")

## 7. View Sample Conversation

In [ ]:
import json
from pathlib import Path

# Load conversation
conv_dir = Path("data/conversations/patient_001/without_history")
conv_file = sorted(conv_dir.glob("day_*.json"))[0]

with open(conv_file, 'r') as f:
    conv = json.load(f)

print("=" * 60)
print(f"SAMPLE CONVERSATION - Day {conv['simulation_day']}")
print("=" * 60)

for i, turn in enumerate(conv['turns'], 1):
    speaker = "PATIENT" if turn['speaker'] == 'patient' else "CHATBOT"
    print(f"\n[{i}] {speaker}:")
    print(turn['message'])

# Show metrics
metrics = conv.get('metrics', {}).get('automatic', {})
print("\n" + "=" * 60)
print("METRICS")
print("=" * 60)
print(f"Questions asked: {metrics.get('bot_questions_asked', 0)}")
print(f"Info gathered: {metrics.get('key_info_gathered', [])}")

## 8. Run Full Experiment (Optional - Takes 30-45 min)

Uncomment and run to do the full 7-day experiment.

In [ ]:
# print("Running FULL experiment: 7 days, both conditions")
# print("This will take 30-45 minutes...\n")
# 
# results = sim.run_full_experiment(
#     patient_ids=["patient_001"],
#     conditions=["without_history", "with_history"],
#     num_days=7
# )
# 
# print("\n✓ Experiment complete!")
# for r in results['results']:
#     print(f"  {r['patient_id']} - {r['condition']}: {r['total_turns']} turns")

## 9. Download Results

In [ ]:
from google.colab import files
import shutil

# Create zip
shutil.make_archive('results', 'zip', 'data')

# Download
files.download('results.zip')

print("✓ Results downloaded!")